In [2]:
import tensorflow as tf
data_dir = "/ssd003/projects/pets/datasets"
dataset = tf.keras.preprocessing.text_dataset_from_directory(f"{data_dir}/yelp/business_federated/7-Eleven/", labels='inferred')

Found 1787 files belonging to 5 classes.


2021-11-01 14:47:52.011915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-01 14:47:52.012183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-01 14:47:52.012377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-01 14:47:52.012564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-11-01 14:47:52.012752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [3]:
data, labels = next(iter(dataset))

In [4]:
data.shape

TensorShape([32])

In [5]:
labels.shape

TensorShape([32])

In [6]:
# As you can see, some of these reviews are... colourful.
data[0]

<tf.Tensor: shape=(), dtype=string, numpy=b"I was just as this establishment (May 20th, around 1:30pm) and the cashier legitimately had to audacity to tell me I shouldn't be wearing an outfit that shows everything & a few other snide remarks. Almost didn't check me out because of it. I'm wearing a fricken blazer and shorts. An outfit I'd wear to my corporate job. Also, the last time I was there, she told me I looked chubby in my ID (Im 105lbs) and almost didn't sell me a pack of smokes. I'm 22. Her attitude was foul and completely inappropriate. She wouldn't give me her name but I sure as hell plan on finding out. To the owner, please respond to this and I will be happy to give you my information. I also will be contacting the corporate office. I would negative stars if I could.">